<a href="https://colab.research.google.com/github/antonellafontanetto/DataScience-Entregas/blob/main/Proyecto_YPF_4x4_AntoV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Proyecto 4x4 YPF: Primer Pilar Vaca Muerta a corto plazo

El siguiente proyecto está focalizado en predecir el comportamiento de determinadas variables en el marco del proyecto 4x4 de YPF, el cual consta de cuadriplicar la empresa en los próximos 4 años a través de la expansión de la producción, eficiencia en los procesos y mayor foco en la generación de valor.

Para lo cual el proyecta va a constar del siguiente procedimiento:

1) Recolección de Datos

2) Exploración y Procesamiento de los datos (exploración de los datos, discreción de variables, normalización, limpieza, visualización previa)

3) Modelado (construir y testear modelos para predecir o clasificar información o encontrar patrones de datos)

4) Puesta en Producción (predecir nuevos datos, comunicar los resultados, integrar los resultados con aplicación)


**1) Recolección de datos**

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [53]:
#Cargo el dataset de Producción de petróleo y gas que vamos a utilizar en el proyecto
produccion = pd.read_csv('/content/produccion.csv', usecols=['anio','empresa','tipopozo','tipoestado','prod_gas','prod_pet','profundidad','areayacimiento','cuenca','provincia','tipo_de_recurso','sub_tipo_recurso'])
produccion


,anio,prod_pet,prod_gas,tipoestado,tipopozo,empresa,profundidad,areayacimiento,cuenca,provincia,tipo_de_recurso,sub_tipo_recurso
0,2018,0.000000,171.937000,Extracción Efectiva,Gasífero,YSUR ENERGÍA ARGENTINA S.R.L.,3828.0,ESTACION FERNANDEZ ORO,NEUQUINA,Rio Negro,NO CONVENCIONAL,TIGHT
1,2017,0.000000,245.850000,Extracción Efectiva,Gasífero,YSUR ENERGÍA ARGENTINA S.R.L.,2338.4,ANTICLINAL CAMPAMENTO SUR,NEUQUINA,Neuquén,NO CONVENCIONAL,TIGHT
2,2018,0.000000,514.696000,Extracción Efectiva,Gasífero,YSUR ENERGÍA ARGENTINA S.R.L.,2707.0,ESTACION FERNANDEZ ORO,NEUQUINA,Rio Negro,NO CONVENCIONAL,TIGHT
3,2015,0.000000,422.110000,Extracción Efectiva,Gasífero,YSUR ENERGÍA ARGENTINA S.R.L.,3305.0,GUANACO,NEUQUINA,Neuquén,NO CONVENCIONAL,TIGHT
4,2018,0.000000,217.103000,Extracción Efectiva,Gasífero,YSUR ENERGÍA ARGENTINA S.R.L.,2618.0,GUANACO,NEUQUINA,Neuquén,NO CONVENCIONAL,TIGHT
...,...,...,...,...,...,...,...,...,...,...,...,...
345377,2024,14.049906,17.378994,Extracción Efectiva,Petrolífero,Petrolera Aconcagua Energia S.A.,2585.0,EL CARACOL,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
345378,2023,0.000000,0.000000,NaN,NaN,Petrolera Aconcagua Energia S.A.,2848.0,CHARCO BAYO,NEUQUINA,Rio Negro,NO CONVENCIONAL,TIGHT
345379,2024,60.960855,602.844409,Extracción Efectiva,Petrolífero,Petrolera Aconcagua Energia S.A.,2840.0,CHARCO BAYO,NEUQUINA,Rio Negro,NO CONVENCIONAL,TIGHT
345380,2024,0.000000,0.000000,Parado Transitoriamente,Gasífero,Petrolera Aconcagua Energia S.A.,2400.0,CHARCO BAYO,NEUQUINA,Rio Negro,NO CONVENCIONAL,TIGHT


In [62]:
#Cargo el dataset de Pozos terminados de petróleo y gas que vamos a utilizar en el proyecto
pozosterminados = pd.read_csv('/content/pozosterminados.csv', usecols=['anio','empresa','areapermisoconcesion','areayacimiento','cuenca','provincia','ubicacion','tipodepozoterminado','concepto'])
pozosterminados

,anio,empresa,areapermisoconcesion,areayacimiento,cuenca,provincia,ubicacion,tipodepozoterminado,concepto
0,2009,ALIANZA PETROLERA ARGENTINA S.A.,CALMUCO - BARREALES COLORADOS,BARREALES COLORADOS,NEUQUINA,Mendoza,On Shore,Explotación,Productivos de Petróleo
1,2009,COMPAÑÍAS ASOCIADAS PETROLERAS S.A.,DIADEMA,DIADEMA,GOLFO SAN JORGE,Chubut,On Shore,Explotación,Productivos de Gas
2,2009,EZ HOLDINGS S.A.,CAÑADON PILAR,CAÑADON PILAR,GOLFO SAN JORGE,Chubut,On Shore,Exploración,Productivos de Petróleo
3,2009,YPF S.A.,CERRO PIEDRA - CERRO GUADAL NORTE,CERRO GUADAL NORTE,GOLFO SAN JORGE,Santa Cruz,On Shore,Servicio,Servicio
4,2009,ALIANZA PETROLERA ARGENTINA S.A.,CALMUCO - BARREALES COLORADOS,BARREALES COLORADOS,NEUQUINA,Mendoza,On Shore,Explotación,Productivos de Gas
...,...,...,...,...,...,...,...,...,...
627603,2019,YPF S.A.,CHIHUIDO DE LA SIERRA NEGRA,CERRITO ALARCON,NEUQUINA,Neuquén,On Shore,Servicio,Servicio
627604,2019,PAN AMERICAN ENERGY SL,ANTICLINAL GRANDE - CERRO DRAGON,TRES PICOS,GOLFO SAN JORGE,Chubut,On Shore,Avanzada,Improductivos
627605,2019,YPF S.A.,LOS CHORRILLOS,LAGUNA CHICA,AUSTRAL,Tierra del Fuego,On Shore,Explotación,Productivos de Gas
627606,2019,PAN AMERICAN ENERGY SL,ANTICLINAL GRANDE - CERRO DRAGON,TRES PICOS,GOLFO SAN JORGE,Chubut,On Shore,Avanzada,Productivos de Petróleo


In [55]:
#Cargo el dataset de etapas fracturadas de pozos petroleros que vamos a utilizar en el proyecto
etapasfracturadas = pd.read_csv('/content/datosdefracturadepozos.csv', usecols=['anio','empresa_informante','areapermisoconcesion','yacimiento','cuenca','tipo_reservorio','subtipo_reservorio','longitud_rama_horizontal_m','cantidad_fracturas','tipo_terminacion','arena_bombeada_nacional_tn','arena_bombeada_importada_tn','agua_inyectada_m3','presion_maxima_psi','potencia_equipos_fractura_hp'])
etapasfracturadas

,cuenca,areapermisoconcesion,yacimiento,tipo_reservorio,subtipo_reservorio,longitud_rama_horizontal_m,cantidad_fracturas,tipo_terminacion,arena_bombeada_nacional_tn,arena_bombeada_importada_tn,agua_inyectada_m3,presion_maxima_psi,potencia_equipos_fractura_hp,empresa_informante,anio
0,NEUQUINA,AGUA DEL CAJON,AGUA DEL CAJON,NO CONVENCIONAL,SHALE,0.0,3,Punzado,0.0000,0.000,2718.20,10190.000000,10897.0,CAPEX S.A.,2019
1,NEUQUINA,AGUA DEL CAJON,AGUA DEL CAJON,NO CONVENCIONAL,SHALE,0.0,1,Punzado,0.0000,0.000,600.00,9250.000000,10251.0,CAPEX S.A.,2018
2,NEUQUINA,AGUADA DE LA ARENA,AGUADA DE LA ARENA,NO CONVENCIONAL,SHALE,1437.3,18,Tapón disparo,3761.3700,536.850,25768.30,15000.000000,32000.0,YPF S.A.,2017
3,NEUQUINA,AGUADA DE LA ARENA,AGUADA DE LA ARENA,NO CONVENCIONAL,SHALE,1518.3,19,Tapón disparo,3903.7050,558.225,27398.37,11348.000000,32000.0,YPF S.A.,2017
4,NEUQUINA,AGUADA DE LA ARENA,AGUADA DE LA ARENA,NO CONVENCIONAL,SHALE,1482.3,19,Tapón disparo,3949.0200,569.925,27157.60,11076.000000,32000.0,YPF S.A.,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4069,NEUQUINA,LA AMARGA CHICA,LA AMARGA CHICA,NO CONVENCIONAL,SHALE,3578.0,57,Tapón disparo,13519.5300,0.000,85810.42,11502.133260,44000.0,YPF S.A.,2025
4070,NEUQUINA,LA AMARGA CHICA,LA AMARGA CHICA,NO CONVENCIONAL,SHALE,3352.0,54,Tapón disparo,12444.4350,0.000,81084.12,12118.146258,44000.0,YPF S.A.,2025
4071,NEUQUINA,LA AMARGA CHICA,LA AMARGA CHICA,NO CONVENCIONAL,SHALE,3398.0,56,Tapón disparo,13340.9295,0.000,85118.19,11361.179928,44000.0,YPF S.A.,2025
4072,NEUQUINA,LOMA CAMPANA,LOMA CAMPANA-LLL,NO CONVENCIONAL,SHALE,2547.7,42,Tapón disparo,10095.5250,0.000,59930.08,12169.208061,44000.0,YPF S.A.,2025


In [56]:
#Filtro por cada dataset solo los registros para YPF S.A.
prod_ypf = produccion[(produccion['empresa'] == 'YPF S.A.')&(produccion['cuenca'] == 'NEUQUINA')&(produccion['anio']>2022)]

prod_ypf


,anio,prod_pet,prod_gas,tipoestado,tipopozo,empresa,profundidad,areayacimiento,cuenca,provincia,tipo_de_recurso,sub_tipo_recurso
8572,2025,32.29,172.10,Extracción Efectiva,Gasífero,YPF S.A.,3844.0,ESTACION FERNANDEZ ORO,NEUQUINA,Rio Negro,NO CONVENCIONAL,TIGHT
8578,2025,0.00,0.00,Abandonado,Otro tipo,YPF S.A.,2827.0,EL OREJANO,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
8584,2025,0.00,0.00,En Estudio,Otro tipo,YPF S.A.,2977.0,LA AMARGA CHICA,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
8590,2025,0.00,0.00,Abandonado,Otro tipo,YPF S.A.,4092.0,EL OREJANO,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
8596,2025,55.99,3.37,Extracción Efectiva,Petrolífero,YPF S.A.,4945.0,LOMA CAMPANA,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
...,...,...,...,...,...,...,...,...,...,...,...,...
199438,2024,0.00,0.00,En Inyección Efectiva,Sumidero,YPF S.A.,2046.0,LOMA COLORADA,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
199440,2024,1642.63,101.18,Extracción Efectiva,Petrolífero,YPF S.A.,5889.0,LOMA COLORADA,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
199442,2024,2441.10,153.31,Extracción Efectiva,Petrolífero,YPF S.A.,0.0,LOMA COLORADA,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
199446,2024,0.00,0.00,Parado Transitoriamente,Gasífero,YPF S.A.,4491.0,LAS TACANAS,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE


In [64]:
pozos_ypf = pozosterminados[(pozosterminados['empresa'] == 'YPF S.A.')&(pozosterminados['cuenca'] == 'NEUQUINA')&(pozosterminados['anio']>2022)]
pozos_ypf

,anio,empresa,areapermisoconcesion,areayacimiento,cuenca,provincia,ubicacion,tipodepozoterminado,concepto


In [59]:
etapasfracturadas_ypf = etapasfracturadas[(etapasfracturadas['empresa_informante'] == 'YPF S.A.')&(etapasfracturadas['cuenca'] == 'NEUQUINA')&(etapasfracturadas['anio']>2022)]
etapasfracturadas_ypf

,cuenca,areapermisoconcesion,yacimiento,tipo_reservorio,subtipo_reservorio,longitud_rama_horizontal_m,cantidad_fracturas,tipo_terminacion,arena_bombeada_nacional_tn,arena_bombeada_importada_tn,agua_inyectada_m3,presion_maxima_psi,potencia_equipos_fractura_hp,empresa_informante,anio
3321,NEUQUINA,RIO NEUQUEN,RIO NEUQUEN,NO CONVENCIONAL,TIGHT,0.00,14,Tapón disparo,0.0000,1538.01,6979.57,10610.045168,24000.0,YPF S.A.,2023
3330,NEUQUINA,LOMA CAMPANA,LOMA CAMPANA,NO CONVENCIONAL,SHALE,2435.00,40,Tapón disparo,9006.6465,0.00,54657.22,12115.159356,40000.0,YPF S.A.,2023
3332,NEUQUINA,LOMA CAMPANA,LOMA CAMPANA,NO CONVENCIONAL,SHALE,2163.21,35,Tapón disparo,8378.8740,0.00,48242.07,11565.996071,40000.0,YPF S.A.,2023
3334,NEUQUINA,LOMA CAMPANA,LOMA CAMPANA,NO CONVENCIONAL,SHALE,1932.00,31,Tapón disparo,7220.8845,0.00,43538.16,11908.067478,40000.0,YPF S.A.,2023
3375,NEUQUINA,LOMA CAMPANA,LOMA CAMPANA-LLL,NO CONVENCIONAL,SHALE,2188.00,36,Tapón disparo,8652.0195,0.00,55748.14,11585.197585,40000.0,YPF S.A.,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4069,NEUQUINA,LA AMARGA CHICA,LA AMARGA CHICA,NO CONVENCIONAL,SHALE,3578.00,57,Tapón disparo,13519.5300,0.00,85810.42,11502.133260,44000.0,YPF S.A.,2025
4070,NEUQUINA,LA AMARGA CHICA,LA AMARGA CHICA,NO CONVENCIONAL,SHALE,3352.00,54,Tapón disparo,12444.4350,0.00,81084.12,12118.146258,44000.0,YPF S.A.,2025
4071,NEUQUINA,LA AMARGA CHICA,LA AMARGA CHICA,NO CONVENCIONAL,SHALE,3398.00,56,Tapón disparo,13340.9295,0.00,85118.19,11361.179928,44000.0,YPF S.A.,2025
4072,NEUQUINA,LOMA CAMPANA,LOMA CAMPANA-LLL,NO CONVENCIONAL,SHALE,2547.70,42,Tapón disparo,10095.5250,0.00,59930.08,12169.208061,44000.0,YPF S.A.,2025


In [60]:
prod_ypf=prod_ypf.rename(columns={'anio':'año'})
prod_ypf

,año,prod_pet,prod_gas,tipoestado,tipopozo,empresa,profundidad,areayacimiento,cuenca,provincia,tipo_de_recurso,sub_tipo_recurso
8572,2025,32.29,172.10,Extracción Efectiva,Gasífero,YPF S.A.,3844.0,ESTACION FERNANDEZ ORO,NEUQUINA,Rio Negro,NO CONVENCIONAL,TIGHT
8578,2025,0.00,0.00,Abandonado,Otro tipo,YPF S.A.,2827.0,EL OREJANO,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
8584,2025,0.00,0.00,En Estudio,Otro tipo,YPF S.A.,2977.0,LA AMARGA CHICA,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
8590,2025,0.00,0.00,Abandonado,Otro tipo,YPF S.A.,4092.0,EL OREJANO,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
8596,2025,55.99,3.37,Extracción Efectiva,Petrolífero,YPF S.A.,4945.0,LOMA CAMPANA,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
...,...,...,...,...,...,...,...,...,...,...,...,...
199438,2024,0.00,0.00,En Inyección Efectiva,Sumidero,YPF S.A.,2046.0,LOMA COLORADA,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
199440,2024,1642.63,101.18,Extracción Efectiva,Petrolífero,YPF S.A.,5889.0,LOMA COLORADA,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
199442,2024,2441.10,153.31,Extracción Efectiva,Petrolífero,YPF S.A.,0.0,LOMA COLORADA,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
199446,2024,0.00,0.00,Parado Transitoriamente,Gasífero,YPF S.A.,4491.0,LAS TACANAS,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE


In [65]:
pozos_ypf=pozos_ypf.rename(columns={'anio':'año'})
pozos_ypf

,año,empresa,areapermisoconcesion,areayacimiento,cuenca,provincia,ubicacion,tipodepozoterminado,concepto
